Stair Tower Built-up Tie Calcs  
E. Durham  4-Aug-2018

Compressive resistance of built-up ties is calculated using CSA S16-14 13.1 with Euler Buckling Stress, $ F_e $ determined used Equ 2-48 from Timoshenko and Gere "Theory of Elastic Stability"

1) CSA S16-14 13.3.4 States "The factored compressive resistance of segmented columns shall be determined using a rational method. Notional loads need not be applied between in-plane lateral supports."

2) It is argued that CSA S16-09 13.3.1 Flexural Buckling of Doubly Symmetric Shapes can be used to determine compressive resistance of segmented columns when the Euler Buckling Stress $ F_e $ in the equation is substituted with critical stress $ \sigma_{cr} $ derived using Equ 2-48 for calculating the critical load $ P_{cr} $ for Bars with Changes in Cross Section as found in Section 2.14 of "Theory of Elastic Stability", 2nd Edition by Timoshenko and Gere. 

3) Equ. 2-48 $ P_{cr} = \frac{m E I_2}{l^2} $ from Timoshenko and Gere "Theory of Elastic Stability"  
where factor $ m $ is taken from Table 2-10.

4) Given that the ratios of $ \frac{I_1}{I_2} $ and $ \frac{a}{l} $ are frequently between the limited values published in Table 2-10, the factor $ m $ is interpolated using 2d splines using the interpolate.bisplev and interpolate.bisplrep functions in the scipy library.

5) Critical stress $ \sigma_{cr} $ substituted for $ F_e $ in CSA S16-09 13.3.1 is derived as follows: $ \sigma_{cr} = \frac{P_{cr}}{A} $  
where $ P_{cr} $ is derived using Equ 2-48 as stated above.


Recap of CSA S16-14 13.3.1 

$ C_r = \frac{\phi A F_y}{(1 + \lambda^{2 n})^{\frac{1}{n}}} $  

where  
- n = 1.34 for hot-rolled, fabricated structural sections and hollow structural sections manufactured in accordance with CSA G40.20, Class C (cold-formed non-stress-relieved), ASTM A500, or ASTM A1085  
- n = 2.24 for doubly symmetric welded three-plate members with flange edges oxy-flame-cut and hollow structural sections manufactured in accordance with CSA G40.20, Class H (hot-formed or cold-formed stress-relieved) and ASTM A1085 with Supplement S1  
- $ \lambda = \sqrt \frac{F_y}{F_e} $  
- where  
- $ F_e = \frac{\pi^2 E}{(\frac{K L}{r})^2} $  

In [1]:
import pint, math
import numpy as np
from scipy import interpolate
unit = pint.UnitRegistry(system='mks')

In [2]:
# define synonyms for common units
inch = unit.inch; mm = unit.mm; m = unit.m; MPa = unit.MPa; psi = unit.psi; kN = unit.kN
ksi = unit.ksi; dimensionless = unit.dimensionless; s = unit.second; kg = unit.kg

In [3]:
%%html
<img src='./img/Fig 2-43.JPG', width=400>

In [4]:
# Case:  
a = 1498 * mm
l = (749*mm) + (1498*mm) + (749*mm)
E = 200000 * MPa
F_y = 317 * MPa
I_1 = 144766 * mm**4  # A500 HSS 1.90" x 0.188" using t_desc = 0.9*t
A_1 = 593.6 * mm**2
I_2 = 437269 * mm**4  # A500 HSS 2.50" x 0.25" using t_desc = 0.9*t
A_2 = 1037.5 * mm**2
r_2 = 20.53 * mm

In [5]:
y_new = round(I_1/I_2,3)
y_new = y_new.magnitude
y_new

0.331

In [6]:
x_new = round(a/l,2)
x_new = x_new.magnitude
x_new

0.5

In [7]:
%%html
<img src='./img/Table 2-10.JPG', width=400>

In [8]:
# Table 2-10
x = np.array([0.2, 0.4, 0.6, 0.8])  # a/l
y = np.array([0.01, 0.1, 0.2, 0.4, 0.6, 0.8])  # I_1/I_2
z = np.array([[0.15, 0.27, 0.60, 2.26],
              [1.47, 2.40, 4.50, 8.59],
              [2.80, 4.22, 6.69, 9.33],
              [5.09, 6.68, 8.51, 9.67],
              [6.98, 8.19, 9.24, 9.78],
              [8.55, 9.18, 9.63, 9.84]])  # factor tabulated by A.N.Dinnik

From the table above, we see that our ratios for $ \frac{I_1}{I_2} $ and $ \frac{a}{l} $ fall between tabulated rows and columns.

Given $ \frac{I_1}{I_2} = 0.33 $ and $ \frac{a}{l} = 0.5 $ we see $ m $ is some value between 4.22 and 8.51

Find $ m $ using 2d splines and the scipy.interpolate library

In [9]:
x, y = np.meshgrid(x, y)
tck = interpolate.bisplrep(x, y, z, s=0)
m = round(interpolate.bisplev(x_new, y_new, tck),3)
print('Given:')
print('I_1 =', I_1, 'I_2 =', I_2, 'a =', a, 'l =', l)
print('m is interpolated as:', m)

Given:
I_1 = 144766 millimeter ** 4 I_2 = 437269 millimeter ** 4 a = 1498 millimeter l = 2996 millimeter
m is interpolated as: 7.007


In [10]:
# Estimate P_cr using Equ 2-48
P_cr = (m*E*I_2)/(l**2)
P_cr.ito(kN)
print('P_cr is estimated as:', round(P_cr,2))

P_cr is estimated as: 68.27 kilonewton


In [11]:
# Derive sigma_cr
sigma_cr = P_cr / A_2
sigma_cr.ito(MPa)
print('sigma_cr is estimated as:', round(sigma_cr,1))

sigma_cr is estimated as: 65.8 megapascal


In [12]:
# Calculate standard value for F_e for comparison with sigma_cr
L = l
K = 1.0 * dimensionless
r = r_2
phi = 0.9
n = 1.34
# Calculate F_e
F_e = (round(math.pi**2,3)*E) / ((K * L)/r)**2
print('F_e =', round(F_e,1) )

F_e = 92.7 megapascal


In [13]:
# Calculate compressive resistance using F_e
lambda_Fe = (F_y/F_e)**(0.5)
C_r_Fe = (phi*A_2*F_y)/((1+(lambda_Fe)**(2*n))**(1/n))
C_r_Fe.ito(kN)
print('Compressive resistance using F_e =', round(C_r_Fe,1))

Compressive resistance using F_e = 75.9 kilonewton


The above value matches the values published in Handbook of Steel Construction, 11th Edition, page 4-100 and would be the correct if the column were composed entirely of HSS 64x6.4. Given our case of build up or segmented columns, compressive resistance is computed to be:

In [14]:
# Calculate compressive resistance using sigma_cr
lambda_cr = (F_y/sigma_cr)**(0.5)
C_r_sigma_cr = (phi*A_2*F_y)/((1+(lambda_cr)**(2*n))**(1/n))
C_r_sigma_cr.ito(kN)
print('Compressive resistance substituting sigma_cr for F_e =', round(C_r_sigma_cr,1))

Compressive resistance substituting sigma_cr for F_e = 56.4 kilonewton
